# Tabula Muris Data: From Raw Data to Analysis

In this notebook, we will go over how to start with the Chan-Zuckerberg's Tabula Muris data from start to finish.

## 1. Downloading Data

The data is public and it can be freely accessed through AWS or the CZ github repo. To download, here we provide two options:

### Option 1 Direct Download from the Chan-Zuckerberg Repo:

Here we will `wget` the data from CZ rep:

In [1]:
!wget https://github.com/chanzuckerberg/scRNA-python-workshop/raw/master/content/data.zip

--2020-10-22 17:44:03--  https://github.com/chanzuckerberg/scRNA-python-workshop/raw/master/content/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chanzuckerberg/scRNA-python-workshop/master/content/data.zip [following]
--2020-10-22 17:44:03--  https://raw.githubusercontent.com/chanzuckerberg/scRNA-python-workshop/master/content/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74104660 (71M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  70.67M   175MB/s    in 0.4s    

2020-10-22 17:44:05 (175 MB/s) - ‘data.zip’ saved [74104660/74104660]



In [2]:
!unzip data.zip

Archive:  data.zip
  inflating: brain_metadata.csv      
  inflating: glioblastoma_normalized.h5ad  
  inflating: glioblastoma_raw.h5ad   
  inflating: brain_counts.csv        
  inflating: pbmc3k.h5ad             


Note: From here on, we will be storing the data in a folder called `TM_Data` 

### Option 2: Download using AWS (recommended)

To make things much nicer, we will first install AWS CLI (command line interface) and then download the data. 

N.b: Having the AWS CLI installed on your machine/cluster will come in very handy!

In [3]:
## get the CLI Zip
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32.3M  100 32.3M    0     0  57.8M      0 --:--:-- --:--:-- --:--:-- 57.7M


Now unzip the file:

In [6]:
# Now unzip and install CLI

### I HAVE AWS INSTALLED SO I GET THE FOLLOWING MESSAGE, BUT YOU SHOULD EXCEPT FULL INSTALLATION
# !unzip awscliv2.zip
!sudo ./aws/install

You can now run: /usr/local/bin/aws --version


Now if you don't have AWS credentials that's fine, we can do a an unauthenticated request (with no sign-in flag) since the data is available publicly

In [7]:
# list the files in the directory
!aws s3 ls s3://czb-tabula-muris/ --no-sign-request

                           PRE 10x_bam_files/
                           PRE facs_bam_files/
2018-10-08 18:38:40  147226550 TM_droplet_mat.csv.gz
2018-10-08 18:38:45  247481595 TM_droplet_mat.h5ad
2018-10-08 18:38:52  245224669 TM_droplet_mat.rds
2018-10-08 18:39:00    6120187 TM_droplet_metadata.csv
2018-10-08 18:39:00  318146414 TM_facs_mat.csv.gz
2018-10-08 18:39:11  487706533 TM_facs_mat.h5ad
2018-10-08 18:39:11  520953990 TM_facs_mat.rds
2018-10-08 18:39:25    5691123 TM_facs_metadata.csv


In [8]:
# here we want to download the files that we have access to, so for example let us download this CSV file

!aws s3 cp s3://czbiohub-tabula-muris/TM_facs_metadata.csv ./TM_Data --no-sign-request

download: s3://czbiohub-tabula-muris/TM_facs_metadata.csv to TM_Data/TM_facs_metadata.csv


## Read in the Data:

Now you can use your favorite way of reading in data to python. In this NB, we will be using Pandas due to the convenience and popularity

In [10]:
import pandas as pd

brain_count = pd.read_csv("./TM_Data/brain_counts.csv", sep = ',', header = "infer", index_col = 0, error_bad_lines = True)

So at this point, we have read in the data which consists of ***unique cells*** (in rows) and ***genes*** (in columns). Let's find how many cells and how many genes we have in the data:

In [11]:
print(f"Number of Unique Cells: {len(brain_count)}")
print(f"Number of Genes: {len(brain_count.columns)}")

Number of Unique Cells: 3401
Number of Genes: 23433


### Getting Information from MetaData

Since we have a meta data associated with the brain cells, let us use it to see what information we can exctract from them

In [12]:
brain_meta = pd.read_csv("./TM_Data/brain_metadata.csv", sep = ',', header = "infer", index_col = 0, error_bad_lines = True)

In [13]:
# print the headers
brain_meta.head(1)

,cell_ontology_class,subtissue,mouse.sex,mouse.id,plate.barcode
cell,,,,,
A1.B003290.3_38_F.1.1,astrocyte,Striatum,F,3_38_F,B003290


We can extract a lot of information from the meta data, for example, let us count the number of subtissues samples in the data, and count the sex of the mice

In [14]:
print(pd.value_counts(brain_meta['subtissue']))
print("----------")
print(pd.value_counts(brain_meta['mouse.sex']))

Cortex         1149
Hippocampus     976
Striatum        723
Cerebellum      553
Name: subtissue, dtype: int64
----------
M    2694
F     707
Name: mouse.sex, dtype: int64


Now that we have successfully loaded the data and are able to read the meta data as well, it is time to move on to annotatint the data

## 2. Data Anotation

We will use `scanpy` to create an `AnnData` object so that we can keep using `scanpy` for further data analysis.

In [15]:
import scanpy as sc

In [16]:
# Here X is going to be the counts, and obs will be the observations or meta data

annotated_data = sc.AnnData(X = brain_count, obs =brain_meta)

In [17]:
# checking if we have the right attributes
annotated_data.obs

,cell_ontology_class,subtissue,mouse.sex,mouse.id,plate.barcode
cell,,,,,
A1.B003290.3_38_F.1.1,astrocyte,Striatum,F,3_38_F,B003290
A1.B003728.3_56_F.1.1,astrocyte,Striatum,F,3_56_F,B003728
A1.MAA000560.3_10_M.1.1,oligodendrocyte,Cortex,M,3_10_M,MAA000560
A1.MAA000564.3_10_M.1.1,endothelial cell,Striatum,M,3_10_M,MAA000564
A1.MAA000923.3_9_M.1.1,astrocyte,Hippocampus,M,3_9_M,MAA000923
...,...,...,...,...,...
P9.MAA000926.3_9_M.1.1,oligodendrocyte precursor cell,Cortex,M,3_9_M,MAA000926
P9.MAA000930.3_8_M.1.1,astrocyte,Cortex,M,3_8_M,MAA000930
P9.MAA000932.3_11_M.1.1,endothelial cell,Hippocampus,M,3_11_M,MAA000932


### Labeling spike-ins 

Since the Tabula Muris data is sequenced through smart-seq, we may have spike-ins*, which we want to annotate seperately. This can be done by assigning genes that have `ERCC` as the `.Var` attribute of the `annodated_data` structure we created.

*What are ***Spike-Ins***? here is the definition from Wikipedia 
An RNA spike-in is an RNA transcript of known sequence and quantity used to calibrate measurements in RNA hybridization assays, such as DNA microarray experiments, RT-qPCR, and RNA-Seq. A spike-in is designed to bind to a DNA molecule with a matching sequence, known as a control probe. This process of specific binding is called hybridization. A known quantity of RNA spike-in is mixed with the experiment sample during preparation. The degree of hybridization between the spike-ins and the control probes is used to normalize the hybridization measurements of the sample RNA.

In [20]:
spike_dict = dict();
spike_counter = 0;

for genes in annotated_data.var_names:
    if 'ERCC' in genes:
        spike_dict[genes] = True;
        spike_counter+=1;
    else:
        spike_dict[genes] = False;        
        
print(f"Found {spike_counter} many Spike-Ins in the data")

Found 92 many Spike-Ins in the data


now that we have a map of every gene for Spike ins as true or false, we can use pd to add nicely add them as an attribute to `annotated_data`

In [21]:
annotated_data.var['ERCC'] = pd.Series(spike_dict);

In [23]:
#and now we want to save the data in the AnnData format

annotated_data.write('./Processed_Data/brain_AnnData.h5ad') ## the h5ad extension is AnnData-specific

... storing 'cell_ontology_class' as categorical
... storing 'subtissue' as categorical
... storing 'mouse.sex' as categorical
... storing 'mouse.id' as categorical
... storing 'plate.barcode' as categorical
